# Alejandro Paredes La Torre,

App to test the images

In [ ]:
# Import necessary modules
import os
import numpy as np
import tensorflow.compat.v1 as tf
from test import generate_images_from_folder, FaceAging  # Import from test.py
from tools.utils import save_images, save_source
from tools.data_generator import ImageDataGenerator
import matplotlib.pyplot as plt

# Disable eager execution for TensorFlow v1 compatibility
tf.disable_eager_execution()

# Configure environment variables for GPU usage
os.environ['CUDA_DEVICES_ORDER'] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'  # Adjust based on your GPU availability

# Define constants (adjust as needed)
batch_size = 1  # Only process one image at a time
image_size = 128
noise_dim = 256
checkpoint_dir = './checkpoints/load_pe/'  # Path to your trained model checkpoints
save_dir = './generated_images/'  # Directory for saving generated images
test_image_path = r"D:\temporal_CV\1. AIPI 590 - Computer Vision\assignment\3. third_project\wiki_preprocess\wiki\74\10016074_1987-07-27_2012.jpg"  # Path to the individual test image

# Configure TensorFlow session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Initialize the data generator
val_generator = ImageDataGenerator(
    batch_size=batch_size,
    height=image_size,
    width=image_size,
    z_dim=noise_dim,
    scale_size=(image_size, image_size),
    shuffle=False,
    mode='test'
)

# Function to process and plot individual images
def generate_and_plot_single_image(image_path):
    # Initialize TensorFlow graph and session
    with tf.Graph().as_default():
        sess = tf.Session(config=config)

        # Initialize the model
        model = FaceAging(
            sess=sess,
            lr=0.001,
            keep_prob=1.0,
            model_num=None,
            batch_size=batch_size,
            age_loss_weight=1.0,
            gan_loss_weight=1.0,
            fea_loss_weight=1.0,
            tv_loss_weight=1.0
        )

        # Define placeholders for the input image and labels
        model.imgs = tf.placeholder(tf.float32, [batch_size, image_size, image_size, 3])
        model.true_label_features_128 = tf.placeholder(
            tf.float32, [batch_size, image_size, image_size, 5]
        )

        # Define the generator operation
        model.ge_samples = model.generate_images(
            model.imgs, model.true_label_features_128, stable_bn=False, mode='test'
        )

        # Load model variables
        model.get_vars()
        model.saver = tf.train.Saver(model.save_g_vars)
        sess.run(tf.global_variables_initializer())

        # Load the pre-trained model
        if model.load(checkpoint_dir, model.saver, 'acgan', 399999):
            print("Model loaded successfully.")
        else:
            print("Model loading failed.")
            return

        # Process the single image
        source, paths = val_generator.load_imgs(image_path, 128)
        temp = np.reshape(source[0], (1, image_size, image_size, 3))

        # Generate the image
        for j in range(1, val_generator.n_classes):
            true_label_fea = val_generator.label_features_128[j]
            feed_dict = {
                model.imgs: temp,
                model.true_label_features_128: true_label_fea,
            }
            samples = sess.run(model.ge_samples, feed_dict=feed_dict)

            # Save the generated image
            output_path = os.path.join(save_dir, f'single_image_{j}.jpg')
            save_images(samples, [1, 1], output_path)

            # Display the image
            print(f"Generated image saved to: {output_path}")
            plt.imshow(samples[0])
            plt.axis('off')
            plt.show()

# Run the function for the single image
generate_and_plot_single_image(test_image_path)

: 

In [ ]:
!"c:/Users/ALEJANDRO/Documents/5. Programming/.virtualenvs/faceAging_venv36/Scripts/python.exe" -m pip install ipykernel -U --force-reinstall'

: 